In [1]:
from dotenv import load_dotenv
load_dotenv()
import os
from openai import OpenAI
from pinecone import Pinecone, ServerlessSpec

/opt/miniconda3/envs/rag/lib/python3.11/site-packages/pinecone/data/index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [2]:
pc = Pinecone(api_key=os.getenv("PINECONE_API_KEY"))
pc.create_index(
    name="rag",
    dimension=1536,
    metric="cosine",
    spec=ServerlessSpec(
        cloud="aws",
        region="us-east-1",
    ),
)

PineconeConfigurationError: You haven't specified an Api-Key.

In [3]:
import json
data = json.load(open('reviews.json'))
data['reviews']

[{'professor': 'Dr. John Smith',
  'subject': 'Computer Science',
  'stars': 4,
  'review': 'Great lecturer, very knowledgeable, but the assignments were tough.'},
 {'professor': 'Dr. Emily Carter',
  'subject': 'Mathematics',
  'stars': 5,
  'review': 'Excellent professor! Made complex topics easy to understand.'},
 {'professor': 'Dr. Rachel Green',
  'subject': 'Psychology',
  'stars': 3,
  'review': 'Engaging but lectures could be more organized.'},
 {'professor': 'Dr. Michael Lee',
  'subject': 'Physics',
  'stars': 2,
  'review': 'Very difficult exams and unclear grading criteria.'},
 {'professor': 'Dr. Susan Harris',
  'subject': 'History',
  'stars': 5,
  'review': 'Passionate and knowledgeable about the subject. Highly recommend!'},
 {'professor': 'Dr. William Brown',
  'subject': 'Economics',
  'stars': 4,
  'review': 'Interesting lectures but too fast-paced at times.'},
 {'professor': 'Dr. Sophia Miller',
  'subject': 'Biology',
  'stars': 3,
  'review': 'Good content, but so

In [4]:
processed_data = []
client = OpenAI()

for review in data['reviews']:
  response = client.embeddings.create(
    input=review['review'],
    model="text-embedding-3-small"
  )
  embedding = response.data[0].embedding
  processed_data.append({
    'values': embedding,
    'id': review['professor'],
    'metadata': {
      'review': review['review'],
      'subject': review['subject'],
      'stars': review['stars'],
    }
  })

OpenAIError: The api_key client option must be set either by passing api_key to the client or by setting the OPENAI_API_KEY environment variable

In [ ]:
processed_data[0]

In [ ]:
index = pc.Index('rag')
index.upsert(
  vectors=processed_data,
  namespace='ns1'
)

In [ ]:
index.describe_index_stats()